# TME 3 : Poids de Hamming et Modèle de Consommation


## Localisation des phases de l'AES et des opérations sur les sous-octets

Dans cette partie préalable, nous allons d'abord essayer d'identifier les différentes phases de l'AES correspondant aux principales fonctions d'un tour : AddRoundKey, SubBytes, ShiftRow et MixColumns. Nous identifierons ensuite le timing des différentes opérations SubBytes pour le premier tour. 

Nous allons travailler avec notre propre AES, donc tout d'abord **téléchargez le fichier `simpleserial-aes-sca.tgz` et placez-le dans le répertoire `hardware/victims/firmware/`**. Vous trouverez ce fichier sur Moodle. Ensuite, exécutez la commande suivante (une seule fois) :

In [ ]:
%%bash
#cd ../hardware/victims/firmware/
#tar -zxvf simpleserial-aes-sca.tgz

Tout d'abord, nous commencerons par examiner l'aspect d'un tour pour le code original. Commençons par compiler l'AES et par programmer la carte cible.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../hardware/victims/firmware/simpleserial-aes-sca/simpleserial-aes-{}.hex".format(PLATFORM)

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes-sca
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
cw.program_target(scope, prog, fw_path)

Pour l'instant, une seule trace est nécessaire :

In [ ]:
# Capture Trace
ktp = cw.ktp.Basic()

key, text = ktp.next() # automated creation of a key and text pair 

scope.adc.samples = 870      # to be adapted if needed (10000 is the default) 870 
scope.adc.offset = 1557       # to be adapted if needed (0 is the default) 1557
trace = cw.capture_trace(scope, target, text, key)

In [ ]:
#offset = scope.adc.offset
#samples = 16*period + 4*mainloop_samples

period = 48
sb_samples = 38
subloop_samples = 10
mainloop_samples = 28

loop = 4*period + 1 * mainloop_samples

def byte_range(b) :
    q = b // 4
    r = b % 4
    s = (q*period + r*loop)
    return s, s+period 
    #return s, s+period
byte_range(15)

Nous pouvons le tracer à l'aide de Bokeh :

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width = 1000, plot_height = 600)

xrange = range(len(trace.wave))

p.line(xrange, trace.wave, line_color = "red")
show(p)

Modifiez maintenant le fichier `hardware/victims/firmware/simpleserial-aes-sca/aes_sca.c` contenant la fonction AES. Ajoutez des nops entre les différentes phases d'un round (comme déjà fait dans TME1, cependant vous trouverez des macros utiles dans le fichier `aes_sca.c`). 

Recompilez, recapturez une trace et identifiez les différentes fonctions. En particulier, vous devez identifier avec certitude le calcul SBox. Notez quelque part comment vous devez ajuster les paramètres `offset` et `samples` de la capture de façon à ce que la trace capturée contienne le calcul SBox complet du premier tour (parce que tous les octets de la clé seront attaqués pendant le TME pendant ce calcul), mais aucune autre partie du calcul (de façon à minimiser le risque d'avoir d'autres échantillons donnant aléatoirement de meilleurs résultats pour une clé erronée que pour la clé correcte). 

Une fois que c'est fait, essayez d'identifier (en utilisant des opérations nop) le décalage et le nombre d'échantillons nécessaires pour capturer chacune des 16 opérations effectuées à l'intérieur du premier calcul SBox. Notez précisément le décalage et le nombre d'échantillons que vous devrez paramétrer pour capturer complètement l'une de ces opérations mais pas les autres.

Les valeurs `offset` et `samples` que vous avez trouvées doivent correspondre à un code dans lequel il n'y a pas d'instruction `nop`.

## Lien entre Consommation Poids de Hamming (HW)

Supprimez le `nops` du code AES si ce n'est pas déjà fait et recompilez l'AES.

Pour revenir à la théorie, la consommation d'énergie sur les lignes de données est fortement corrélée au changement d'état des éléments matériels (lignes de données, registres). Comme ces changements d'état ne sont pas faciles à caractériser, nous pouvons utiliser le fait que souvent, dans la pratique, cela se traduit par une corrélation entre la valeur des lignes de données et la consommation d'énergie mesurée (en raison des transitions avec des valeurs constantes). C'est ce que l'on peut voir dans l'illustration suivante :

![Lignes de puissance](img/dpa_4bits_powerhw_scaled.png)

Comment prouver que cela est vrai ? Traçons le poids de Hamming (HW) des données pour le savoir, ainsi que les courbes de puissance. Celui-ci est défini par le nombre de bits à 1 dans un mot. Nous allons pour cela utiliser l'algorithme AES, parce que nous venons de l'analyser et parce que c'est la cible de l'attaque DPA que nous effectuerons à la fin de ce TME.

## Capture des Traces de Consommation

La capture des traces de puissance sera très similaire aux tutoriels précédents, sauf que cette fois nous utiliserons une boucle pour capturer plusieurs traces, ainsi que numpy pour les stocker. Vous trouverez facilement sur Internet de la documentation sur numpy (tableaux numpy, opérations sur les tableaux numpy, etc.) si nécessaire. 

Ce n'est pas nécessaire, mais nous allons aussi afficher la trace que nous obtenons en utilisant `bokeh`.

### Capture des traces

Ci-dessous, vous pouvez voir la boucle de capture. Le corps principal de la boucle charge un nouveau texte en clair, arme le scope, envoie la clé et le texte en clair, puis finalement enregistre et ajoute notre nouvelle trace à la liste `traces[]`. À la fin, nous convertissons les données de la trace en tableaux numpy, puisque c'est ce que nous utiliserons pour l'analyse.

In [ ]:
# Capture Traces
from tqdm.notebook import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic() # object dedicated to the generation of key (fixed by default) and plain text

traces = [] # list of traces
N = 5000  # Number of traces

for i in tnrange(N, desc = 'Capturing traces'):
    key, text = ktp.next()  # creation of a pair comprising (fixed) key and text 

    
    trace = cw.capture_trace(scope, target, text, key) # a trace is composed of the following fields :
                                                       #    a wave (samples)
                                                       #    textin (input text), textout (output text)
                                                       #    key (input key)
    
    if trace is None:
        continue
    traces.append(trace)  

# Convert traces to numpy arrays 
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])    # for fixed key generation, these keys are all the same


Maintenant que nous avons nos traces, nous pouvons également les tracer à l'aide de Bokeh :

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width = 1000, plot_height = 300)

xrange = range(len(traces[0].wave))
p.line(xrange, traces[0].wave, line_color = "red")
show(p)

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analyse des Traces

### Utilisation des données des traces

Maintenant que nous avons des traces, regardons ce que nous avons enregistré. En regardant les parties précédentes du script, nous pouvons voir que les données de la trace sont dans `trace_array`, tandis que `textin_array` stocke ce que nous avons envoyé à notre cible pour être crypté. Pour l'instant, obtenons quelques informations de base (le nombre total de traces, ainsi que le nombre de points d'échantillonnage dans chaque trace) sur les traces, puisque nous en aurons besoin plus tard :

In [ ]:
numtraces = np.shape(trace_array)[0] # total number of traces
numpoints = np.shape(trace_array)[1] # samples per trace
print(numtraces, numpoints)
print(key.hex())

Pour l'analyse, nous devrons parcourir en boucle chaque octet de la clé que nous voulons attaquer, ainsi que chaque trace :
```python
for bnum in range(0, 16):
    for tnum in range(0, numtraces):
        pass
```
Bien que nous n'ayons pas fait de boucle, notez que chaque trace est constituée d'un ensemble de points d'échantillonnage.
Examinons de plus près l'AES afin de pouvoir remplacer ce `pass` par du code réel.

### Calcul du Poids de  Hamming des données

Maintenant que nous disposons de quelques traces de puissance de notre cible que nous pouvons utiliser, nous pouvons passer aux étapes suivantes de l'attaque. En revenant sur le fonctionnement de l'AES, nous nous souvenons que nous essayons en fait de cibler la position en bas de cette figure :

![S-Box HW Leakage Point](img/Sbox_cpa_detail.png)

L'objectif est donc de déterminer la sortie de la SBox, qui est définie comme suit :

In [ ]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)



Nous devons donc écrire une fonction prenant un seul octet d'entrée, un seul octet de clé, et renvoyant la sortie de la S-Box :

In [ ]:
def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

Enfin, n'oubliez pas que nous voulons connaître le poids de Hamming de la sortie de la S-Box. Nous supposons que le système fuit le poids de Hamming de la sortie de cette S-Box. Nous supposons que le système divulgue le poids de Hamming de la sortie de cette S-Box. Comme solution naïve, nous pourrions d'abord convertir chaque nombre en binaire et compter les 1 :

```python
>>> bin(0x1F)
'0b11111'
>>> bin(0x1F).count('1')
5
```

Cela sera cependant assez lent. À la place, nous créons une table de recherche nommée `HW` en utilisant cette idée :

In [ ]:
HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]

# Example - PlainText is 0x12, key is 0xAB
HW[intermediate(0x12, 0xAB)]

### Affichage du Poids de Hamming

Enfin, nous allons associer chaque trace à une classe qui correspond au HW de la sortie de la S-box. Ensuite, nous pouvons représenter chacune des différentes "classes" dans une couleur différente. Nous devrions ainsi voir s'il y a un endroit où la différence de poids de Hamming est relativement évidente, c'est-à-dire s'il y a un endroit où la consommation et le HW semblent être corrélés. Nous y parvenons facilement en utilisant le tableau `HW` et la fonction `intermediate()` que nous avons définis plus tôt, ainsi qu'une boucle pour tracer toutes les traces.

Pour voir facilement l'emplacement, nous pouvons zoomer sur une zone spécifique des traces. Dans l'exemple suivant, seul un petit sous-ensemble de la capture complète sera tracé. Nous savons déjà ce qu'est un "bon" point en considérant la partie de la trace qui correspond au calcul de S-Box pour un octet donné de la clé (vous avez déterminé ces parties au début de ce TME). Vous devrez adapter les `plot_start` et `plot_end` en utilisant vos découvertes dans les étapes pré-requises. Vous devez tester avec tous les octets de clé (i.e. octet de clé 0, octet de clé 1, ..., octet de clé 15).

Au bon échantillon, vous devriez observer une gradation régulière du rouge (de clair à foncé), car ces couleurs sont associées à la valeur HW de la valeur ciblée. 

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer
import numpy as np

output_notebook()
p = figure(plot_width = 1000, plot_height = 400) 

# Must run S-Box() script first to define the HW[] array and intermediate() function
offset = 20
bnum = 11              # set bnum to play with different key byte (from 0 to 15) 
start, end = byte_range(bnum)
plot_start = start + offset      # set here the sample where to start for the selected key byte
plot_end = end + offset          # set here the sample where to end for the selected key byte

xrange = range(len(traces[0].wave))[plot_start:plot_end]

color_mapper = (brewer['RdPu'][9])
print(color_mapper)

for trace in traces:
    hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
    p.line(xrange, trace.wave[plot_start:plot_end], line_color=color_mapper[hw_of_byte])

show(p)

# b0 : 49-50
# b4 : 96-97
# b8 : 144-145
# b12 : 220-221

# b1 : 269-270
# b5 : 316-317
# b9 : 364-365
# b13 : 440-441

# b2 : 488-489
# b6 : 536-537
# b10 : 584-585
# b14 : 660-661

# b3 : 708-709
# b7 : 756-757
# b11 : 804-805
# b15 : 836-837

b = [49, 269, 488, 708, 96, 316, 536, 756, 144, 364, 584, 804, 220, 440, 660, 836]

for i in range(16) : 
    print("byte", i, byte_range(i))

### Calcul des moyennes aux points d'intérêt

En partant de l'idée qu'il existe des différences, traçons-les pour voir à quel point elles sont "linéaires" dans la réalité. Nous allons choisir un point (à nouveau) et l'utiliser pour obtenir les moyennes. La procédure suivante permet de trouver et d'imprimer les moyennes :

In [ ]:
import numpy as np

bnum = 15
avg_point = 837
hw_list = [[], [], [], [], [], [], [], [], []]
for trace in traces:
    hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
    hw_list[hw_of_byte].append(trace.wave[avg_point])
hw_mean_list = [np.mean(hw_list[i]) for i in range(0, 9)]

for hw in range(0, 9):
    print("HW " + str(hw) + ": " + str(hw_mean_list[hw]))

Si vous avez le bon point `avg_point`, ce qui précède devrait être à peu près linéaire.
Traçons un beau graphique pour le voir visuellement :

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(title="HW vs Voltage Measurement")
p.line(range(0, 9), hw_mean_list[0:9], line_color="red")
p.xaxis.axis_label = "Hamming Weight of Intermediate Value"
p.yaxis.axis_label = "Average Value of Measurement"
show(p)

In [ ]:
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_svg, export_png
from bokeh.palettes import inferno, viridis
from bokeh.palettes import all_palettes
from bokeh.models import ColorBar, LogTicker
from selenium import webdriver
import chromedriver_binary

b = [49, 269, 488, 708, 96, 316, 536, 756, 144, 364, 584, 804, 220, 440, 660, 836]

output_notebook()
p = figure(title="HW vs Voltage Measurement")

#color_mapper = bokeh.palettes.mpl['Plasma'][16]
color_mapper = (inferno(16))
#color_mapper = (viridis(16))
print(color_mapper)

for bnum in range(0, 16) :
    avg_point = b[bnum]
    hw_list = [[], [], [], [], [], [], [], [], []]
    for trace in traces:
        hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
        hw_list[hw_of_byte].append(trace.wave[avg_point])
    hw_mean_list = [np.mean(hw_list[i]) for i in range(0, 9)]

    #for hw in range(0, 9):
        #print("HW " + str(hw) + ": " + str(hw_mean_list[hw]))
    p.line(range(0, 9), hw_mean_list[0:9], line_color=color_mapper[bnum], legend_label=str(bnum))
p.xaxis.axis_label = "Hamming Weight of Intermediate Value"
p.yaxis.axis_label = "Average Value of Measurement"
p.legend.title = "Byte #"

#p.output_backend = "svg"
#export_svg(p, filename="hw_voltage_measurement.svg")
#export_png(p, filename="hw_voltage_measurement.png", width=2400, height=1000)

show(p)

Voilà, c'est fait ! Vous devriez obtenir une belle droite (ou presque). Si ce n'est pas le cas, il se peut que vous ayez sélectionné le mauvais point, modifiez-le si nécessaire. Quoi qu'il en soit, vous remarquerez peut-être que la pente est opposée à celle à laquelle vous vous attendiez.

Il y a une bonne raison à cela. Si vous vous souvenez de la façon dont nous mesurons le courant dans l'appareil, vous constaterez que la tension diminuera pour une AUGMENTATION du courant. Vous pouvez le voir dans la figure suivante :

![Point de mesure](img/vmeasure.png)

Nous mesurons la chute aux bornes de la résistance shunt. Une augmentation du courant entraîne une augmentation de la tension aux bornes de la résistance. Lorsqu'il n'y a pas de courant, il n'y a pas de chute aux bornes de la résistance. Mais comme nous ne mesurons qu'une seule extrémité de la résistance, nous observons une tension plus élevée en l'absence de courant.

Nous pouvons corriger la pente en inversant simplement le sens de la mesure (en ajoutant un - devant la mesure).